In [15]:
import pandas as pd

billboard_df = pd.read_csv("billboard_100.csv")

genius_characters = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789- '

def format_for_genius(string):
    formatted_string = ''
    for character in string:
        if character in genius_characters:
            formatted_string += character
        elif character == '&':
            formatted_string += 'and'
    return formatted_string

def remove_features(string):
    for i in range(len(string)):
        if string[i:i+9] == 'featuring' or string[i:i+4] == 'with':
            string = string[:i-1]
    return string

def format_genius_lyrics(string):
    lyrics_started = False
    formatted_string = ''
    bracket_open = False

    for i in range(len(string)):
        # The string Lyricsgenius returns starts with some text about the
        # number of contributors and '[Title] lyrics', so this code makes 
        # sure it doesn't start reading until after it sees that word.
        if string[i-6:i] == 'Lyrics':
            lyrics_started = True
        # Adds all text not in brackets
        if lyrics_started is True:
            if string[i] == '[':
                bracket_open = True
            if bracket_open is False:
                formatted_string += string[i]
            if string[i] == ']':
                bracket_open = False
            if string[i+1:i+6] == 'Embed':
                lyrics_started = False

    # LyricsGenius sometimes leaves a random string of numbers right
    # before the Embed at the end, so this makes sure we get rid of those
    while True:
        if formatted_string[-1] not in '1234567890':
            break
        formatted_string = formatted_string[:-1]
    # Turn newlines into spaces to improve formatting for sentiment analysis
    formatted_string = formatted_string.replace('\n', ' ')
    
    return formatted_string





title = []
artists = []

for i in range(len(x)):
    title.append(format_for_genius(x['Title'][i]))
    artists.append(format_for_genius(remove_features(x['Artist(s)'][i])))

title_series = pd.Series(title)
artists_series = pd.Series(artists)
# '''
lyrics = []

from lyricsgenius import Genius
import api_key
import time

token = api_key.client_access_token

genius = Genius(token)

for i in range(5):
    song = genius.search_song(title_series[i], artists_series[i])
    time.sleep(.5)
    lyrics.append(format_genius_lyrics(song.lyrics))

df = pd.DataFrame({'No.':billboard_df['No.'][:5], 'Title': title_series[:5], 'Artists': artists_series[:5], 'Year': billboard_df['Year'][:5], 'Lyrics': pd.Series(lyrics)})

# print(df)
# '''    
df.to_csv('billboard_with_lyrics', encoding='utf-8', index=False)    






Searching for "Thrift Shop" by Macklemore and Ryan Lewis...
Done.
Searching for "Blurred Lines" by Robin Thicke...
Done.
Searching for "Radioactive" by Imagine Dragons...
Done.
Searching for "Harlem Shake" by Baauer...
Done.
Searching for "Cant Hold Us" by Macklemore and Ryan Lewis...
Done.


In [17]:
x = pd.read_csv('billboard_with_lyrics')
print(x)

   No.          Title                    Artists    Year  \
0    1    Thrift Shop  Macklemore and Ryan Lewis  2013.0   
1    2  Blurred Lines               Robin Thicke  2013.0   
2    3    Radioactive            Imagine Dragons  2013.0   
3    4   Harlem Shake                     Baauer  2013.0   
4    5   Cant Hold Us  Macklemore and Ryan Lewis  2013.0   

                                              Lyrics  
0   Hey, Macklemore, can we go thrift shopping? W...  
1   Everybody get up Everybody get up Hey, hey, h...  
2   Woah, oh-oh Woah, oh-oh Woah, oh-oh Woah   I'...  
3   Con los terroristas, -tas, -tas, -tas -tas -t...  
4   Hey, hey, hey Good to see you Come on, dude, ...  
